In [1]:
from langchain_gigachat.chat_models.gigachat import GigaChat
from typing import List, Optional, Dict, Any

from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableLambda
import os
import json

In [2]:
class Message(BaseModel):
    role: str
    content: str

class Thread(BaseModel):
    id: str
    messages: List[Message]

class Retrieval(BaseModel):
    type: str  # Тип контента: 'text' или 'image'
    content: str  # Текст или ссылка на изображение
    source: str  # Ссылка на источник данных: txt или pdf
    page: Optional[int]  # Номер страницы в pdf
    relevance: float  # Релевантность контента

class ModelInput(BaseModel):
    thread: Thread
    retrievals: List[Retrieval]

class ModelResponse(BaseModel):
    message: str  # Сообщение для отправки пользователю
    retrievals: List[Retrieval]

In [3]:
llm = GigaChat(
    credentials="MTEwMzY1YmEtMzYzMy00YWQ1LThmMTQtNWEzODM0NzUwN2IwOjU2ZDlkOGY1LWU0MDUtNDQ2Ni1hNTQyLWU5N2M4MzJmY2FmOA==",
    verify_ssl_certs=False,
    timeout=6000,
    model="GigaChat-preview",
    base_url="https://gigachat-preview.devices.sberbank.ru/api/v1"
)

In [4]:
# Папка для сохранения диалогов
CHAT_DIR = "chats"
if not os.path.exists(CHAT_DIR):
    os.makedirs(CHAT_DIR)

# Словарь для хранения памяти чатов по id
chat_memories: Dict[str, ConversationBufferMemory] = {}

def save_chat_history(chat_id: str, thread: Thread):
    """Сохраняет историю чата в формате JSON."""
    chat_file = os.path.join(CHAT_DIR, f"{chat_id}.json")
    with open(chat_file, 'w', encoding='utf-8') as f:
        json.dump(thread.dict(), f, ensure_ascii=False, indent=2)

def load_chat_history(chat_id: str) -> Thread:
    """Загружает историю чата из файла JSON, если он существует."""
    chat_file = os.path.join(CHAT_DIR, f"{chat_id}.json")
    if os.path.exists(chat_file):
        with open(chat_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            thread = Thread(**data)
    else:
        thread = Thread(id=chat_id, messages=[])
    return thread

def handle_message(chat_id: str, user_input: str) -> str:
    # Загружаем или создаем новый Thread
    thread = load_chat_history(chat_id)
    
    # Добавляем сообщение пользователя в историю
    user_message = Message(role="user", content=user_input)
    thread.messages.append(user_message)
    
    # Подготавливаем историю сообщений для модели
    messages = []
    for msg in thread.messages:
        if msg.role == "user":
            messages.append(HumanMessage(content=msg.content))
        elif msg.role == "assistant":
            messages.append(AIMessage(content=msg.content))
        # Можно добавить обработку других ролей, если необходимо

    # Генерируем ответ модели
    assistant_response = llm.predict_messages(messages).content

    # Здесь можно добавить логику для получения retrievals
    # Например, вызвать функцию, которая возвращает список Retrieval
    # Для примера добавим пустой список retrievals
    retrievals = []  # Замените на вашу реализацию получения retrievals

    # Добавляем ответ ассистента в историю
    assistant_message = Message(
        role="assistant",
        content=assistant_response,
        retrievals=retrievals if retrievals else None
    )
    thread.messages.append(assistant_message)

    # Сохраняем обновленную историю чата
    save_chat_history(chat_id, thread)

    # Возвращаем ответ ассистента
    return assistant_response

# Пример использования
if __name__ == "__main__":
    chat_id = "thread_001"
    user_message = "Can you provide insights on this topic?"
    assistant_response = handle_message(chat_id, user_message)
    print(f"Assistant: {assistant_response}")

    # Следующее сообщение в том же чате
    user_message = "Can you elaborate on the trends mentioned?"
    assistant_response = handle_message(chat_id, user_message)
    print(f"Assistant: {assistant_response}")

    # Еще одно сообщение
    user_message = "What are some specific use cases in healthcare?"
    assistant_response = handle_message(chat_id, user_message)
    print(f"Assistant: {assistant_response}")

C:\Users\Sergey\AppData\Local\Temp\ipykernel_28256\1577899394.py:44: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  assistant_response = llm.predict_messages(messages).content


Assistant: Sure! I'd be glad to help with that. Could you please tell me what specific topic or question you're interested in so I can give more tailored insights?
Assistant: Absolutely! Let’s dive into some of the most prominent tech trends shaping various industries today.

### 1. **Artificial Intelligence (AI) and Machine Learning**
   - **Growth Drivers**: The rapid advancement in AI algorithms and computing power has made it possible for machines to perform complex tasks like image recognition, natural language processing, and decision-making. This is driving innovation across sectors such as healthcare, finance, manufacturing, and autonomous vehicles.
   - **Impact**: Companies are leveraging AI to automate processes, improve customer service through chatbots, predict market trends, enhance cybersecurity, and develop personalized products and services.
   
### 2. **Internet of Things (IoT)**
   - **Connectivity Expansion**: IoT devices—ranging from smart home appliances to indust

In [4]:
# Настраиваем парсер
parser = PydanticOutputParser(pydantic_object=ModelResponse)

# Функция для подготовки сообщений из URL
def _get_messages_from_input(model_input: ModelInput):
    thread_messages = [
        HumanMessage(content=msg.content)
        if msg.role == 'user' else
        HumanMessage(content=msg.content, role='assistant')
        for msg in model_input.thread.messages
    ]
    return {
        "history": thread_messages,
        "retrievals": model_input.retrievals
    }

# Создаем шаблон подсказки (prompt)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Вы — помощник, который обрабатывает запросы пользователя на основе следующих правил:\n"
            "1) Если запрос пользователя — вопросительный, ответьте на вопрос.\n"
            "2) Если запрос пользователя — поисковый, опишите содержимое на картинке или в тексте из RAG.\n"
            "3) Если в данных из RAG нет нужных данных, сообщите пользователю, что нужные данные не найдены.\n\n"
            "Пожалуйста, предоставьте ваш ответ в следующем формате JSON:\n{format_instructions}"
        ),
        MessagesPlaceholder("history"),
    ]
).partial(format_instructions=parser.get_format_instructions())

# Определяем цепочку
chain = (
    RunnableLambda(_get_messages_from_input)
    | prompt
    | llm
    | parser
)

# Пример использования
# Загружаем файлы (замените на ваши реальные файлы)
file1 = llm.upload_file(open("chart1.jpg", "rb"))
file2 = llm.upload_file(open("chart2.jpg", "rb"))
file3 = llm.upload_file(open("chart3.jpg", "rb"))

# Создаем ModelInput с сообщениями и retrievals
model_input_example = ModelInput(
    thread=Thread(
        id="thread_123",
        messages=[
            Message(role="user", content="Что изображено на этой диаграмме?"),
        ]
    ),
    retrievals=[
        Retrieval(
            type="image",
            content=file1.id_,  # Используем ID загруженного файла
            source="chart1.jpg",
            page=None,
            relevance=0.9
        )
    ]
)

# Генерируем ответ
response = chain.invoke(model_input_example)

# Выводим результат
print(response.message)
for retrieval in response.retrievals:
    print(f"Retrieval: {retrieval}")

На этой диаграмме изображен процесс обработки запросов пользователя. Диаграмма показывает, как система обрабатывает различные типы запросов, включая вопросительные, поисковые и другие виды запросов, и какие действия выполняются на каждом этапе.
